# Problem klasyfikacji gęstości (DCP)

### Imports

In [1]:
# pip install joblib

In [2]:
import math
import numpy as np
from operator import itemgetter
from joblib import Parallel, delayed

Funkcje generujące diagram czasoprzestrzenny:

In [3]:
def eca_get_lut(rule_num: int) -> np.ndarray:
    return np.array([int(x) for x in bin(rule_num)[2:].zfill(8)], dtype=np.uint8)

def eca_evolve(lut: np.ndarray, x: np.ndarray) -> np.ndarray:
    return lut[7 - (np.roll(x, 1) * 4 + x * 2 + np.roll(x, -1))]

def eca_evolve_spacetime(lut: np.ndarray, initial_conf: np.ndarray, steps: int) -> np.ndarray:
    rows = [initial_conf]
    for _ in range(1, steps):
        rows.append(eca_evolve(lut, rows[-1]))
    return np.stack(rows)


Funkcja do losowania konfiguracji początkowej:

In [4]:
def generate_random_initial_configuration(length: int) -> np.ndarray:
    return np.random.randint(2, size=length)

## Wczytywanie konfiguracji pana Marcina
Pliki z konfiguracjami można pobrać z https://github.com/D3M80L/CA/tree/master/phd/data.

Funkcja konwertująca ciąg bajtów na binarną reprezentację. Zwraca tablicę N pierwszych bitów:

In [5]:
def convert_configuration(configuration: bytes, N: int):
    binary_str = ''.join(f'{byte:08b}' for byte in configuration)
    return np.array([int(bit) for bit in binary_str[:N]])


Funkcja wczytująca wszystkie konfiguracje z pliku do pamięci operacyjnej:

In [6]:
def configuration_reader(file_name: str, N: int, negate: bool):
    bytes_per_configuration = math.ceil(N / 8)
    configurations = []

    with open(file_name, 'rb') as file:
        while True:
            bytes_read = file.read(bytes_per_configuration)
            if not bytes_read:
                break

            cfg = convert_configuration(bytes_read, N)
            configurations.append(cfg)
            
            if negate:
                configurations.append(1 - cfg)

    return np.stack(configurations)


- Pierwszy argument (str): nazwa pliku, zktórego chcemy pobrać konfiguracje.
- Drugi argument (int): długość konfiguracji zapisanych w pliku.
- Trzeci argument (bool): czy w wynikowej liście mają znaleźć się też "zanegowane" konfiguracje; False - konfiguracje będą miały gęstość mniejszą niż 0.5; True - konfiguracje, których gęstość jest mniejsza jaki i większa od 0.5.

Przykład użycia:

In [7]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
configurations2 = configuration_reader("ALL_N21.bin", 21, False)

In [8]:
print(configurations[0:10])
print(configurations2[0:10])

[[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 1]]


---

# zad.7
Dla ustalonego N, przebadaj wszystkie ECAs dając im limit czasu i wylicz dla jakiego % wszystkich konfiguracji początkowych otrzymaną poprawną klasyfikację gęstości. Zaprezentuj wyniki w postaci listy par (numer reguły, % poprawnych klasyfikacji), np.

(0, 50%)\
...\
(255, 50%)

Znajdź / wyróżnij regułę, która ma największy wśród ECAs % poprawnych klasyfikacji.

In [9]:
# gęstość konfiguracji po N krokach porównujemy z gęstością konfiguracji początkowej
# konfiguracje początkowe do badania weźmiemy z pliku ALL_N21.bin
# limit czasu to maksymalna długość kroków w diagramie czasoprzestrzennym (umownie 2x długość konfiguracji początkowej)

In [10]:
def dcp(lut, initial_config, steps):
    initial_density = sum(initial_config)/len(initial_config)
    result_config = eca_evolve_spacetime(lut, initial_config, steps)[-1]
    
    if initial_density < 0.5:
        if np.all(result_config == 0):
            return 1
        else:
            return 0
    if initial_density > 0.5:
        if np.all(result_config == 1):
            return 1
        else:
            return 0
    return 0


In [11]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
len(configurations)

99880

In [14]:
results = []
for rule in range(256):
    print(f'Rule {rule}...')
    lut = eca_get_lut(rule)
    # l = [dcp(lut, config, 42) for config in configurations]
    l = Parallel(n_jobs=-1)(delayed(dcp)(lut, config, 42) for config in configurations)
    results.append((rule, round(100*sum(l)/len(configurations),3)))
    print(f'({results[-1][0]}, {results[-1][1]}%)')

Rule 0...
(0, 50.0%)
Rule 1...
(1, 1.025%)
Rule 2...
(2, 0.001%)
Rule 3...
(3, 0.0%)
Rule 4...
(4, 1.392%)
Rule 5...
(5, 0.0%)
Rule 6...
(6, 0.002%)
Rule 7...
(7, 0.0%)
Rule 8...
(8, 50.0%)
Rule 9...
(9, 0.02%)
Rule 10...
(10, 0.001%)
Rule 11...
(11, 0.0%)
Rule 12...
(12, 0.001%)
Rule 13...
(13, 0.0%)
Rule 14...
(14, 0.001%)
Rule 15...
(15, 0.0%)
Rule 16...
(16, 0.001%)
Rule 17...
(17, 0.0%)
Rule 18...
(18, 0.167%)
Rule 19...
(19, 0.0%)
Rule 20...
(20, 0.002%)
Rule 21...
(21, 0.0%)
Rule 22...
(22, 0.817%)
Rule 23...
(23, 0.0%)
Rule 24...
(24, 0.001%)
Rule 25...
(25, 0.0%)
Rule 26...
(26, 0.001%)
Rule 27...
(27, 0.0%)
Rule 28...
(28, 0.001%)
Rule 29...
(29, 0.0%)
Rule 30...
(30, 0.002%)
Rule 31...
(31, 0.0%)
Rule 32...
(32, 50.0%)
Rule 33...
(33, 0.056%)
Rule 34...
(34, 0.001%)
Rule 35...
(35, 0.0%)
Rule 36...
(36, 11.759%)
Rule 37...
(37, 0.101%)
Rule 38...
(38, 0.001%)
Rule 39...
(39, 0.0%)
Rule 40...
(40, 50.0%)
Rule 41...
(41, 0.001%)
Rule 42...
(42, 0.001%)
Rule 43...
(43, 0.0%)
Ru

In [15]:
sorted(results, key=itemgetter(1))

[(3, 0.0),
 (5, 0.0),
 (7, 0.0),
 (11, 0.0),
 (13, 0.0),
 (15, 0.0),
 (17, 0.0),
 (19, 0.0),
 (21, 0.0),
 (23, 0.0),
 (25, 0.0),
 (27, 0.0),
 (29, 0.0),
 (31, 0.0),
 (35, 0.0),
 (39, 0.0),
 (43, 0.0),
 (45, 0.0),
 (47, 0.0),
 (49, 0.0),
 (51, 0.0),
 (53, 0.0),
 (55, 0.0),
 (57, 0.0),
 (59, 0.0),
 (61, 0.0),
 (63, 0.0),
 (67, 0.0),
 (69, 0.0),
 (71, 0.0),
 (75, 0.0),
 (77, 0.0),
 (79, 0.0),
 (81, 0.0),
 (83, 0.0),
 (85, 0.0),
 (87, 0.0),
 (89, 0.0),
 (93, 0.0),
 (95, 0.0),
 (99, 0.0),
 (101, 0.0),
 (103, 0.0),
 (113, 0.0),
 (115, 0.0),
 (117, 0.0),
 (119, 0.0),
 (2, 0.001),
 (10, 0.001),
 (12, 0.001),
 (14, 0.001),
 (16, 0.001),
 (24, 0.001),
 (26, 0.001),
 (28, 0.001),
 (34, 0.001),
 (38, 0.001),
 (41, 0.001),
 (42, 0.001),
 (44, 0.001),
 (46, 0.001),
 (48, 0.001),
 (50, 0.001),
 (52, 0.001),
 (56, 0.001),
 (58, 0.001),
 (60, 0.001),
 (66, 0.001),
 (68, 0.001),
 (70, 0.001),
 (74, 0.001),
 (76, 0.001),
 (78, 0.001),
 (80, 0.001),
 (82, 0.001),
 (84, 0.001),
 (88, 0.001),
 (90, 0.001),


In [17]:
sorted(results, key=itemgetter(1), reverse=True)[0:12]

[(128, 50.001),
 (136, 50.001),
 (160, 50.001),
 (168, 50.001),
 (192, 50.001),
 (224, 50.001),
 (234, 50.001),
 (238, 50.001),
 (248, 50.001),
 (250, 50.001),
 (252, 50.001),
 (254, 50.001)]

# zad.8
Henryk Fukś pokazał, że idealne rozwiązanie DCP można uzyskać stosując ECA 184 przez "połowę czasu", a następnie ECA 232 przez następną 
połowę czasu.

Stosując wybrany "zbiór Marcina" lub zbiór wygenerowany losowo sprawdź, czy odkrycie prof. Fuksia daje się odtworzyć eksperymentalnie.

**Wariant (*)** - czy poza ECA 184 istnieją inne ECAs, które w powiązaniu z ECA 232 dają taką własność? Chodzi o inną parę ECA x, ECA 232 lub ewentualnie ECA x, ECA y, które stosowane tak jak przez prof. Fuksia rozwiązują DCP dokładnie.

## 184 - traffic right and 232 - majority

In [11]:
def dcp2(lut1, lut2, initial_config, steps):
    initial_density = sum(initial_config)/len(initial_config)
    config = eca_evolve_spacetime(lut1, initial_config, steps//2)[-1]
    result_config = eca_evolve_spacetime(lut2, config, steps//2)[-1]

    if initial_density < 0.5:
        if np.all(result_config == 0):
            return 1
        else:
            return 0
    if initial_density > 0.5:
        if np.all(result_config == 1):
            return 1
        else:
            return 0
    return 0


In [12]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
lut184 = eca_get_lut(184)
lut232 = eca_get_lut(232)
# l = [dcp2(lut184, lut232, config, 42) for config in configurations]
l = Parallel(n_jobs=-1)(delayed(dcp2)(lut184, lut232, config, 42) for config in configurations)
print(f'Poprawna klasyfikacja gęstości: {round(100*sum(l)/len(configurations),3)}%')

Poprawna klasyfikacja gęstości: 100.0%


## 226 - traffic left and 232 - majority

In [13]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
lut226 = eca_get_lut(226)
lut232 = eca_get_lut(232)
# l = [dcp2(lut226, lut232, config, 42) for config in configurations]
l = Parallel(n_jobs=-1)(delayed(dcp2)(lut226, lut232, config, 42) for config in configurations)
print(f'Poprawna klasyfikacja gęstości: {round(100*sum(l)/len(configurations),3)}%')

Poprawna klasyfikacja gęstości: 100.0%


## 170 - left shift and 232 - majority

In [18]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
lut170 = eca_get_lut(170)
lut232 = eca_get_lut(232)
# l = [dcp2(lut226, lut232, config, 42) for config in configurations]
l = Parallel(n_jobs=-1)(delayed(dcp2)(lut170, lut232, config, 42) for config in configurations)
print(f'Poprawna klasyfikacja gęstości: {round(100*sum(l)/len(configurations),3)}%')

Poprawna klasyfikacja gęstości: 89.63%


## 204 - identity and 232 - majority

In [19]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
lut204 = eca_get_lut(204)
lut232 = eca_get_lut(232)
# l = [dcp2(lut226, lut232, config, 42) for config in configurations]
l = Parallel(n_jobs=-1)(delayed(dcp2)(lut204, lut232, config, 42) for config in configurations)
print(f'Poprawna klasyfikacja gęstości: {round(100*sum(l)/len(configurations),3)}%')

Poprawna klasyfikacja gęstości: 89.63%


## 240 - right shift and 232 - majority

In [20]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
lut240 = eca_get_lut(240)
lut232 = eca_get_lut(232)
# l = [dcp2(lut226, lut232, config, 42) for config in configurations]
l = Parallel(n_jobs=-1)(delayed(dcp2)(lut240, lut232, config, 42) for config in configurations)
print(f'Poprawna klasyfikacja gęstości: {round(100*sum(l)/len(configurations),3)}%')

Poprawna klasyfikacja gęstości: 89.63%


# zad.9 (zmodyfikowane - patrz lab5.ipynb)
<s>Zapoznaj się z wynikami z artykułu: *Mendonça, J. R. G. (2019).* **Simply modified GKL density classifiers that reach consensus faster.** *Physics Letters A, 383(19), 2264–2266.* doi:10.1016/j.physleta.2019.04.033 (https://scihub.se/https://www.sciencedirect.com/science/article/abs/pii/S0375960119303512?via=ihub).</s>

<s>Spróbuj powtórzyć eksperymenty tam wykonane dla wybranego N dla którego mamy "zbiór Marcina" badając średni czas klasyfikacji oraz % poprawnych klasyfikacji dla reguł GKL(j, k) dla kilku przypadków j, k.</s>

<s>Otrzymane wyniki porównaj z tymi z Zadania 7. To znaczy jak bardzo lepsze są najlepsze reguły GKL, które uda Ci się znaleźć w porównaniu do najlepszych ECAs.</s>

# <del>zad.10(*)</del>
<del>Sprawdź jak wybór zbioru testowych warunków początkowych wypływa na wyniki eksperymentów z Zadania 8.</del>

<del>Czy jesteś w stanie znaleźć przykład parametrów N (liczba komórek), num_conf (liczba losowanych warunków w zbiorze), p (prawdopodobieństwo 0 w 
komórce) do losowania, aby pokazać, że eliminacja duplikatów (shift-equal) ma znaczenie - tj. niechlujne dobranie "losowego" zbioru warunków początkowych zawyża / zaniża szacowaną jakość klasyfikacji.</del>

<del>**Podpowiedź**: Możesz porównać wyniki otrzymane na wybranym "zbiorze Marcina" z wynikami na zbiorze otrzymanym w zwykłym losowaniu dla konkretnego N.</del>